~ 1.5 hour runtime

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON, SPARQLExceptions
import pandas as pd
from typing import List
from tqdm.notebook import tqdm
import time

In [2]:
result_lst = []

In [ ]:
categories_to_scan = ['wd:Q38723', 'wd:Q189004', 'wd:Q3918', 'wd:Q62078547', 'wd:Q269770', 'wd:Q12241709', 'wd:Q428602', 'wd:Q9826', 'wd:Q64063386', 'wd:Q3914', 'wd:Q2385804', 'wd:Q5341295', 'wd:Q902104', 'wd:Q1377182', 'wd:Q6540832', 'wd:Q494230', 'wd:Q1321960', 'wd:Q1143635', 'wd:Q20820271', 'wd:Q1663017', 'wd:Q423208', 'wd:Q11391922', 'wd:Q352102', 'wd:Q149566', 'wd:Q875538', 'wd:Q159334']

avg_len = 3
remainder = 0
sub_arrays = []
start = 0

for i in range(9):
    end = start + avg_len + (1 if i < remainder else 0)
    sub_arrays.append(categories_to_scan[start:end])
    start = end

categories = sub_arrays
print('running on categories:', categories)

In [4]:
def fetch_wikidata_entities(file):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    
    for categories_subset in tqdm(categories[:-1], desc="Processing Categories"):
        query = f"""
            SELECT ?institution ?instance WHERE {{
                VALUES ?instance {{{' '.join(categories_subset)}}}
                {{
                    ?institution wdt:P31 ?instance .
                }}
                UNION
                # Second layer: Subclasses of instances (subclass of a subclass)
                {{
                    ?subclass wdt:P279* ?instance .
                    ?institution wdt:P31* ?subclass .
                }}
                
                OPTIONAL {{ ?institution wdt:P17 ?country . }}
                FILTER (!BOUND(?country) || ?country IN (wd:Q30))
            }}

            GROUP BY ?institution ?instance
        """

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        try:
            # Execute the query
            results = sparql.query().convert()['results']['bindings']
            result_lst.extend(results)
            for result in results:
                file.write(str(result))

            print(f'adding {len(results)} entries to result_lst')
            print(len(result_lst))
            # Process the results
        
        except Exception as e:
            print(f"An error occurred: {e}")
    return result_lst
    

In [5]:
def fetch_wikidata_entities_subset(file, categories_subset):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    
    for category in tqdm(categories_subset, desc="Processing Categories"):
        print(category)
        query = f"""
            SELECT ?institution ?instance WHERE {{
            VALUES ?instance {{{category}}}

            ?institution wdt:P31 ?instance .
            
            OPTIONAL {{ ?institution wdt:P17 ?country . }}
            FILTER (!BOUND(?country) || ?country IN (wd:Q30))
            }}

            GROUP BY ?institution ?instance
        """

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        try:
            # Execute the query
            results = sparql.query().convert()['results']['bindings']
            result_lst.extend(results)
            for result in results:
                file.write(str(result))

            print(f'adding {len(results)} entries to result_lst')
            print(len(result_lst))
            # Process the results
        
        except Exception as e:
            print(f"An error occurred: {e}")
    return result_lst
    

In [6]:
file_name = f'wikidata_entites_{time.time()}.txt'

In [ ]:
with open(file_name, 'w') as file:
    df_entities = fetch_wikidata_entities(file)

In [ ]:
# Run on any categories that timed out
category = categories[-1]

with open(file_name, 'a') as file:
    df_entities = fetch_wikidata_entities_subset(file, category)

In [9]:
df_entities = pd.DataFrame(df_entities)

In [ ]:
df_entities.shape

In [11]:
df_entities["institution"] = df_entities["institution"].astype(str)
df_entities["institution_hash"] = df_entities["institution"].apply(hash)
df_unique = df_entities.drop_duplicates(subset="institution_hash")

In [ ]:
# remove duplicate ids
print(df_unique.shape)
df_unique.sample(5)

In [13]:
df_entities = df_unique

In [15]:
df_entities['institution_id'] = df_entities['institution'].apply(lambda x: x.split('/')[-1][:-2])

# Write the extracted IDs to a text file
output_file = "institution_ids.txt"
with open(output_file, "w") as f:
    f.writelines(f"{inst_id}\n" for inst_id in df_entities['institution_id'])

# Get each entity

In [3]:
def process_entity(results: List[dict], entity_id):
    """
    Aggregates the input data into a dictionary where the key is valueLabel['value'] 
    and the value is a dictionary mapping propertyLabel['value'] to valueLabel['value'].

    Args:
        data (list of dict): List of dictionaries containing 'valueLabel' and 'propertyLabel'.

    Returns:
        dict: Aggregated dictionary.
    """
    aggregated = {}

    if results:
        alt_labels = []  # To store alternative labels
        parent_orgs = []  # To store alternative labels

        for item in results:
            # Process valueLabel and propertyLabel
            if "valueLabel" in item and "propertyLabel" in item:
                value_label = item["valueLabel"]["value"]
                property_label = item["propertyLabel"]["value"]

                if property_label not in aggregated:
                    aggregated[property_label] = [value_label]
                else:
                    aggregated[property_label].append(value_label)
            
            # Collect altLabels
            if "parentOrg" in item and "parentOrgLabel" in item:
                parent_orgs.append({
                    "id": item["parentOrg"]["value"].split("/")[-1],  # Extract Wikidata ID
                    "name": item["parentOrgLabel"]["value"]
                })

            # Collect Wikipedia URL
            if "wikipediaURL" in item:
                aggregated['wikipediaURL'] = item["wikipediaURL"]["value"]
            if "wikipediaURL" in item:
                aggregated['wikipediaURL'] = item["wikipediaURL"]["value"]

        # Add alternative labels to the dictionary
        if alt_labels:
            aggregated['altLabels'] = list(set(alt_labels))  # Remove duplicates if any

        # Add parent organizations to the dictionary
        if parent_orgs:
            aggregated['parentOrganizations'] = parent_orgs

        aggregated['wd_id'] = entity_id
        aggregated['name'] = results[0]['itemLabel']['value']
        return aggregated
    else:
        print("no results found")

In [4]:
def get_entity(match):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    query = f"""
        SELECT ?itemLabel ?propertyLabel ?valueLabel ?altLabel ?wikipediaURL ?parentOrg ?parentOrgLabel WHERE {{
            BIND(wd:{match} AS ?item)  # Bind the input entity to ?item
            ?item ?prop ?value .
            ?property wikibase:directClaim ?prop .

            OPTIONAL {{
                ?value rdfs:label ?valueLabel .
                FILTER (lang(?valueLabel) = "en")
            }}

            OPTIONAL {{ ?item skos:altLabel ?altLabel . FILTER (LANG(?altLabel) = "en") }}

            OPTIONAL {{
                ?wikipediaURL schema:about ?item ;
                            schema:inLanguage "en" ;
                            schema:isPartOf <https://en.wikipedia.org/>.
            }}

            OPTIONAL {{
                ?item wdt:P749 ?parentOrg .
                ?parentOrg rdfs:label ?parentOrgLabel .
                FILTER (LANG(?parentOrgLabel) = "en")
            }}

            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    results = sparql.query().convert()
    results = results['results']['bindings']
    return results

# Execute

In [5]:
import time

def fetch_entity_with_retry(entity, max_retries=5, backoff_factor=1.5):
    retries = 0
    while retries < max_retries:
        try:
            # Replace this with the actual API call
            result = get_entity(entity)  # Your function that fetches the entity
            return result
        except SPARQLExceptions.QueryBadFormed as e:
            print("Bad query format:", e)
            break
        except Exception as e:
            # Check for Retry-After header in response
            if e == 'HTTP Error 429: Too Many Requests':
                retry_after = 5
                if retry_after:
                    wait_time = int(retry_after)
                    print(f"Rate limit hit. Retrying after {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Retry-After header not found. Retrying after 5 seconds...")
                    time.sleep(5)
            else:
                print(f"Unexpected error: {e}")
                break
        # except Exception as e:
        #     # If a 429 error occurs, handle it
        #         retries += 1
        #         wait_time = backoff_factor ** retries + random.uniform(0, 4)
        #         print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")
        #         time.sleep(wait_time)
    print(f"Max retries reached. Skipping {entity}")
    return None  # Return None if retries are exhausted


In [ ]:
# ~ 1h20m

all_res = []
output_txt = f"raw_wiki_entities_{time.time()}.txt"
print(f'writing entities to {output_txt}')

with open('id_failures.txt', 'r') as file:
    # Read all lines and store them in a list
    lines = file.readlines()

# Strip the newline characters from each line
results = [line.strip() for line in lines]
if results:
    for entity_id in tqdm(results, desc="Processing Entities", unit="entity", miniters=100, mininterval=0.5):
        entity_data = fetch_entity_with_retry(entity_id)  # Retry logic added here
        if entity_data:
            entity_data = process_entity(entity_data, entity_id)  # Process the entity after fetching
            with open(output_txt, mode="a", encoding="utf-8") as txtfile:
                # Format the entity data as a string for writing
                txtfile.write(f"{str(entity_data)}\n")
            
            all_res.append(entity_data)

In [2]:
import ast
import json
import pandas as pd


file_path = 'raw_wiki_entities_12_6.txt'
data = []

# Open the file and read line by line
with open(file_path, 'r') as file:
    for i, line in enumerate(file):
        try:
            # Safely evaluate the Python-like dictionary format
            record = ast.literal_eval(line.strip())
            data.append(record)
        except json.JSONDecodeError as e:
            print(f"Error on line {i}: {line.strip()}")
            print(f"JSONDecodeError: {e}")

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                     coordinate location  \
0      [Point(-88.55 47.12), Point(-88.55 47.12), Poi...   
1      [Point(-72.527777777 42.388888888), Point(-72....   
2      [Point(-71.038445 42.313432), Point(-71.038445...   
3      [Point(-72.730555555 41.781111111), Point(-72....   
4      [Point(-96.580833333 39.191666666), Point(-96....   
...                                                  ...   
76483                                                NaN   
76484                                                NaN   
76485                                                NaN   
76486                 [Point(-104.81916667 41.14472222)]   
76487                                                NaN   

                                            wikipediaURL  \
0      https://en.wikipedia.org/wiki/Michigan_Technol...   
1      https://en.wikipedia.org/wiki/University_of_Ma...   
2      https://en.wikipedia.org/wiki/University_of_Ma...   
3      https://en.wikipedia.org/wiki/Un

In [3]:
df.sample()

,coordinate location,wikipediaURL,Commons category,NLA Trove people ID,endowment,VIAF ID,Microsoft Academic ID,subreddit,HAL structure ID,Integrated Postsecondary Education Data System ID,...,proxy,Wikipedia Library partner ID,payload mass,theorised by,PubMed publication ID,author name string,Gateway to Research Project ID,principal investigator,funding scheme,Dictionary of Archives Terminology ID
31115,NaN,https://en.wikipedia.org/wiki/The_Post_Oak_School,[The Post Oak School],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
num_columns = df.shape[1]
print(num_columns)
df.columns

1512


Index(['coordinate location', 'wikipediaURL', 'Commons category',
       'NLA Trove people ID', 'endowment', 'VIAF ID', 'Microsoft Academic ID',
       'subreddit', 'HAL structure ID',
       'Integrated Postsecondary Education Data System ID',
       ...
       'proxy', 'Wikipedia Library partner ID', 'payload mass', 'theorised by',
       'PubMed publication ID', 'author name string',
       'Gateway to Research Project ID', 'principal investigator',
       'funding scheme', 'Dictionary of Archives Terminology ID'],
      dtype='object', length=1512)

# Get ids that were lost to 429 errors

In [ ]:
import ast
import json


institutions_file = 'institution_ids.txt'
output_file = "missing_ids.txt"

# Read institutions.txt
with open(institutions_file, "r") as f:
    institution_ids = {line.strip() for line in f.readlines()}

# Read processed_entities.txt and extract 'wd_id' values
processed_ids = set()
with open(processed_entities_file, "r") as f:
    for line in f:
        try:
            entity = ast.literal_eval(line.strip())
            
            # Check if 'wd_id' exists and process it
            if isinstance(entity, dict) and 'wd_id' in entity:
                processed_ids.add(entity['wd_id'])
        except json.JSONDecodeError:
            print(f"Error decoding line: {line.strip()}")  # Optional: handle or log malformed lines

# Compute the difference
missing_ids = institution_ids - processed_ids

# Write missing IDs to output file
with open(output_file, "w") as f:
    f.writelines(f"{wd_id}\n" for wd_id in missing_ids)

print(f"Missing IDs written to {output_file}")

In [ ]:
output_txt = f"processed_entities_{time.time()}.txt"
print(f'writing entities to {output_txt}')

# results = fetch_wikidata_entities()
results = missing_ids
if results:
    for entity_id in tqdm(results, desc="Processing Entities", unit="entity"):
        entity_data = fetch_entity_with_retry(entity_id)  # Retry logic added here
        if entity_data:
            entity_data = process_entity(entity_data, entity_id)  # Process the entity after fetching
            with open(output_txt, mode="a", encoding="utf-8") as txtfile:
                # Format the entity data as a string for writing
                txtfile.write(f"{str(entity_data)}\n")
            
            all_res.append(entity_data)

# Process data

In [ ]:
df = pd.DataFrame(all_res)

In [ ]:
df.sample()

In [7]:
num_rows = df.shape[0]
print(num_rows)

76488


In [8]:
# Count rows before filtering
initial_count = len(df)

# Drop schools without names
df.drop(df[df['name'] == df['wd_id']].index, inplace=True)

# Count rows after filtering
final_count = len(df)

# Calculate and print the number of dropped rows
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} items without a name")

Dropping 17286 items without a name


In [9]:
# Count rows before filtering
initial_count = len(df)

# Drop items without instance of
df = df.dropna(subset=['instance of'])

# Count rows after filtering
final_count = len(df)

# Calculate and print the number of dropped rows
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} items without an instance of. {final_count} entities remaining.")

Dropping 929 items without an instance of. 58273 entities remaining.


In [10]:
# drop all columns where one or zero rows have value

initial_count = df.shape[1]

df = df.dropna(axis=1, how='all')
df = df.loc[:, df.notna().sum() > 1]
df = df[['name'] + ['wd_id'] + [col for col in df.columns if col != 'name' and col != 'wd_id']]

final_count = df.shape[1]
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} columns without any content")

Dropping 464 columns without any content


In [11]:
df.sample(5)

,name,wd_id,coordinate location,wikipediaURL,Commons category,NLA Trove people ID,endowment,VIAF ID,Microsoft Academic ID,subreddit,...,French national research structure ID,size designation,ICANNWiki page ID,editor,WikiProjectMed ID,SlideShare username,IFTTT service ID,APA Dictionary of Psychology entry,NCI Thesaurus ID,J. Paul Getty Museum agent ID
15958,Holy Family Catholic High School,Q5885434,[Point(-93.6442 44.8585)],https://en.wikipedia.org/wiki/Holy_Family_Cath...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8898,Elmezzi Graduate School for Molecular Medicine,Q5366732,[Point(-73.702321 40.775742)],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3097,Berkeley College,Q2897191,"[Point(-73.9794 40.754), Point(-73.9794 40.754...",https://en.wikipedia.org/wiki/Berkeley_College,"[Berkeley College, Berkeley College, Berkeley ...",NaN,NaN,NaN,"[134446601, 134446601, 134446601, 134446601, 1...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46063,Mackinac College,Q6724494,[Point(-84.6054 45.8514)],https://en.wikipedia.org/wiki/Mackinac_College,[Mackinac College],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41199,University of North Carolina at Chapel Hill De...,Q101016019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
with open('columns.txt', 'w') as f:
    for column in df.columns:
        f.write(column + '\n')

In [15]:
pd.set_option('display.max_rows', None)
print(df['instance of'].value_counts())

# Write df to file

In [13]:
import time

file_name = f'processed_schools_{time.time()}.csv'

df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

Wrote df to processed_schools_1733495318.135554.csv


In [ ]:
# Explode the 'instance of' column to turn lists into separate rows
df_exploded = df.explode('instance of')

# Count the occurrences of each value in 'instance of'
instance_counts = df_exploded['instance of'].value_counts()

# Display the counts
print(instance_counts)

In [ ]:
df_instance_counts = pd.DataFrame(instance_counts)

In [ ]:
with open('instance_counts.txt', 'w') as file:
    for value, count in instance_counts.items():
        file.write(f"{value}: {count}\n")

In [12]:
# remove all rows with certain values for instanceof
instances_to_remove = ['organization', 'botanical garden', 'laboratory', 'corporate research group', 'neural buoyancy pool', 'online database', 'human', 'academic library']

In [29]:
def get_nicknames(id):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    query = f"""
        SELECT ?altLabel
        WHERE {{
            wd:{id} skos:altLabel ?altLabel.
            FILTER (LANG(?altLabel) = "en")
        }}
        """
    
    try:
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        results = sparql.query().convert()
        results = results['results']['bindings']
        results = [result['altLabel']['value'] for result in results]
        return results
    except Exception as e:
        print(e, id)

In [31]:
from tqdm import tqdm
import pandas as pd

# Enable tqdm for pandas
tqdm.pandas()

# Apply the function with a progress bar
df['altLabels'] = df.progress_apply(lambda row: get_nicknames(row['wd_id']), axis=1)

  3%|▎         | 1480/58273 [03:54<2:33:54,  6.15it/s] 

HTTP Error 429: Too Many Requests Q16959503


  3%|▎         | 1484/58273 [03:54<1:54:50,  8.24it/s]

HTTP Error 429: Too Many Requests Q16967741
HTTP Error 429: Too Many Requests Q16968025


  3%|▎         | 1491/58273 [03:55<2:07:43,  7.41it/s]

...

 96%|█████████▋| 56221/58273 [2:25:59<04:27,  7.67it/s]

HTTP Error 429: Too Many Requests Q3480396


 96%|█████████▋| 56224/58273 [2:25:59<04:00,  8.51it/s]

HTTP Error 429: Too Many Requests Q11421146


100%|██████████| 58273/58273 [2:30:33<00:00,  6.45it/s]


In [32]:
df.sample(10)

,name,wd_id,coordinate location,wikipediaURL,Commons category,NLA Trove people ID,endowment,VIAF ID,Microsoft Academic ID,subreddit,...,size designation,ICANNWiki page ID,editor,WikiProjectMed ID,SlideShare username,IFTTT service ID,APA Dictionary of Psychology entry,NCI Thesaurus ID,J. Paul Getty Museum agent ID,altLabels
148,"University of Maryland, Baltimore County",Q735049,"[Point(-76.711255555 39.2555), Point(-76.71125...",https://en.wikipedia.org/wiki/University_of_Ma...,"[University of Maryland, Baltimore County, Uni...",NaN,NaN,"[145133422, 145133422]","[79272384, 79272384]","[UMBC, UMBC]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[UMBC, umbc.edu]"
12453,Millard South High School,Q3314250,[Point(-96.1467 41.2042)],https://en.wikipedia.org/wiki/Millard_South_Hi...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
20049,Morgantown High School,Q6911982,[Point(-79.9564 39.6242)],https://en.wikipedia.org/wiki/Morgantown_High_...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
19333,Norwalk High School,Q7060785,[Point(-73.389444444 41.121944444)],https://en.wikipedia.org/wiki/Norwalk_High_Sch...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
29290,Marvell Academy,Q6778225,[Point(-90.9122 34.5699)],https://en.wikipedia.org/wiki/Marvell_Academy,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
48814,Anderson County Schools,Q104876885,NaN,https://en.wikipedia.org/wiki/Anderson_County_...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
50815,New Hartford Senior High School,Q128700100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[New Hartford High School]
51115,Hudson Gardens,Q5928661,[Point(-105.023 39.6046)],https://en.wikipedia.org/wiki/Hudson_Gardens,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
14753,Dallastown Area High School,Q5211460,[Point(-76.653 39.893)],https://en.wikipedia.org/wiki/Dallastown_Area_...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
16697,Greater Hartford Academy of Mathematics and Sc...,Q5600568,[Point(-72.6836 41.7506)],https://en.wikipedia.org/wiki/Greater_Hartford...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [33]:
cols_to_keep = ["name",
    "wd_id",
    "wikipediaURL",
    "short name",
    "nickname",
    "altLabels",
    "alternative name",
    "count of students",
    "image",
    "seal image",
    "logo image",
    "small logo or icon",
    "icon",
    "coat of arms image",
    "X username",
    "Instagram username",
    "Facebook username",
    "member of",
    "instance of",
    "street address",
    "coordinate location",
    "headquarters location",
    "postal code",
    "located in the administrative territorial entity",
    "location",
    "country",
    "official website",
    "LinkedIn company or organization ID",
    "has subsidiary",
    "part of",
    "parent organization",
    "owner of",
    "parentOrganizations",
    "official name",
    "has part(s)",
    "described at URL",
    "subclass of",
    "IRS Employer Identification Number",
    "school district",
    "start grade",
    "end grade"]
subset_df = df[cols_to_keep]

In [41]:
# Consolidate nickname colummns
subset_df['nicknames'] = subset_df.apply(lambda row: row.get("altLabels", '') + row.get("short name", '') + row.get("nickname", '') + row.get("alternative name", ''))
subset_df.drop("altLabels", axis = 1)
subset_df.drop("short name", axis = 1)
subset_df.drop("nickname", axis = 1)
subset_df.drop("alternative name", axis = 1)
    
# Consolidate image columns
subset_df['icon'] = subset_df.apply(lambda row: row.get("seal image", '') + row.get("logo image", '') + row.get("small logo or icon", '') + row.get("icon", '') + row.get("coat of arms image", ''))
subset_df.drop("seal image", axis = 1)
subset_df.drop("logo image", axis = 1)
subset_df.drop("small logo or icon", axis = 1)
subset_df.drop("icon", axis = 1)
subset_df.drop("coat of arms image", axis = 1)
    
# Consolidate location columns
subset_df['location'] = subset_df.apply(lambda row: row.get("location", '') + row.get("headquarters location", '') + row.get("located in the administrative territorial entity", ''))
subset_df.drop("location", axis = 1)
subset_df.drop("headquarters location", axis = 1)
subset_df.drop("located in the administrative territorial entity", axis = 1)
subset_df.drop("location", axis = 1)

,name,wd_id,wikipediaURL,short name,nickname,altLabels,alternative name,count of students,image,seal image,...,official name,has part(s),described at URL,subclass of,IRS Employer Identification Number,school district,start grade,end grade,nicknames,icon
0,Michigan Technological University,Q12432,https://en.wikipedia.org/wiki/Michigan_Technol...,NaN,NaN,"[MTU, Mich Tech, Michigan Tech, Michigan Tech ...",NaN,"[7074, 7074, 7074, 7074, 7074, 7074, 7074]",[http://commons.wikimedia.org/wiki/Special:Fil...,[http://commons.wikimedia.org/wiki/Special:Fil...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,University of Massachusetts Amherst,Q15142,https://en.wikipedia.org/wiki/University_of_Ma...,NaN,NaN,"[Massachusetts Agricultural College, Massachus...",NaN,"[24209, 24209, 24209, 24209, 24209, 31350, 313...",[http://commons.wikimedia.org/wiki/Special:Fil...,[http://commons.wikimedia.org/wiki/Special:Fil...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,University of Massachusetts Boston,Q15144,https://en.wikipedia.org/wiki/University_of_Ma...,"[UMass Boston, UMass Boston, UMass Boston]",NaN,"[UMass Boston, umb.edu, University of Massachu...",NaN,"[16259, 16259, 16259]",[http://commons.wikimedia.org/wiki/Special:Fil...,NaN,...,"[University of Massachusetts Boston, Universit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Saint Joseph,Q24399,https://en.wikipedia.org/wiki/University_of_Sa...,NaN,NaN,"[USJ, Saint Joseph College]",NaN,"[2305, 2305]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kansas State University,Q31249,https://en.wikipedia.org/wiki/Kansas_State_Uni...,NaN,NaN,"[KSU, K-State, Kansas Agricultural College, Ka...",NaN,"[20854, 20854, 20854, 20854, 20854, 20854, 168...",[http://commons.wikimedia.org/wiki/Special:Fil...,NaN,...,[Kansas State University of Agriculture and Ap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76483,1st Engineer Brigade,Q27951753,https://en.wikipedia.org/wiki/1st_Engineer_Bri...,NaN,NaN,[1st Engineer Special Brigade],NaN,NaN,[http://commons.wikimedia.org/wiki/Special:Fil...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76484,23rd Quartermaster Brigade,Q55734210,NaN,NaN,NaN,"[23d Quartermaster Group, 23d Quartermaster Re...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76485,1st Aviation Brigade,Q91439560,https://en.wikipedia.org/wiki/1st_Aviation_Bri...,NaN,NaN,[],NaN,NaN,[http://commons.wikimedia.org/wiki/Special:Fil...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76486,Cheyenne High School,Q30610476,https://en.wikipedia.org/wiki/Cheyenne_High_Sc...,NaN,NaN,[],NaN,NaN,[http://commons.wikimedia.org/wiki/Special:Fil...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
subset_df.shape

(58273, 39)

In [101]:
# remove all rows with certain values for instanceof
instances_to_remove = ['organization', 'botanical garden','lyceum','degree program','church building','historic district','academic department of the Spanish language','building', 'laboratory', 'corporate research group', 'legal clinic','type of university','Q56377753','experimental station','research institute of international relations', 'China-focused think tank','international organization','501(c)(4) organization','environmental organization','secret service','foundation','Native American organization','business incubator','company','polling organization',"Revelation in Mormonism", 'bus type',"commandment", 'subsidiary','body of knowledge','government-organized non-governmental organization','land trust','peace organization','Islamic organization','research reactor','research center','geospatial laboratory', 'mixed martial arts training facility', 'laboratory', 'first-order class', 'training brigade','music library', 'publishing company', 'aspect in a geographic region', 'language regulator', 'nonprofit organization', 'think tank', 'neural buoyancy pool', 'school building', 'online database', 'human', 'academic library', 'academic department','academic library','seminary','academic department','think tank','interdisciplinary program','education company','open-access publisher','university campus','school accreditor','type of educational institution','university museum','university research group','business','Catholic seminary','arboretum','dental school','residential college','public research university','music school','military school','charter school','school for the deaf','arboretum','department','university building','library','US state education agency','school district of Oregon','state university system','yeshiva','government agency','library network','501(c)(3) organization','gymnasium','Colleges That Change Lives','Syracuse University Centers and Institutes','dance studio','research library','board of directors','hospital','museum','geography department','division of an educational institution','academic publisher','library building','university network','military unit','art museum','division','facility','fixed construction','statistical service','arts educational institution','library branch','professional association','archive','park','historic building','training facility','educational website','rose garden','university art museum','university library','board of education','teaching hospital','community college district','architectural structure','research unit','type of Roman Catholic institution','research department','degree program','academy of sciences', 'website','zoo', 'United States federal agency', 'military facility', 'language regulator', 'advocacy group', 'Christian organization', 'department of education', 'Wikimedia disambiguation page']

In [102]:
subset_df['instance of'].value_counts()

instance of
high school                                                                                     11471
university                                                                                       6480
school district                                                                                  5569
school                                                                                           4520
[public school, high school]                                                                     2480
                                                                                                ...  
[primary school, high school, private school]                                                       1
[vocational-technical school, engineering school, magnet school, public school, high school]        1
[high school, engineering school, charter school network, educational institution]                  1
[high school, special education]                                      

In [103]:
subset_df['instance of'] = subset_df['instance of'].apply(lambda instances: instances if isinstance(instances, list) else [instances])

subset_df['instance of'] = subset_df['instance of'].apply(
    lambda instances: [instance for instance in instances if instance not in instances_to_remove]
)

In [104]:
# Count rows before filtering
initial_count = len(subset_df)

# Drop items without instance of
subset_df = subset_df.dropna(subset=['instance of'])
subset_df = subset_df[subset_df['instance of'].apply(lambda x: x != [])]  # Drop rows where 'instance of' is an empty list
# Count rows after filtering
final_count = len(subset_df)

# Calculate and print the number of dropped rows
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} items without an instance of. {final_count} entities remaining.")

Dropping 33 items without an instance of. 55177 entities remaining.


In [ ]:
def schoolTypeMap(name: str):
    name = name.casefold()
    types = []
    if 'university' in name:
        types.append('university')
    if 'college' in name:
        types.append('college')
    if 'prep' in name:
        types.append('high school')
    if 'business' in name:
        types.append('business school')
    if 'law' in name:
        types.append('law school')
    if 'tech' in name or 'engineer' in name:
        types.append('engineering school')
    if 'elementary' in name:
        types.append('elementary school')
    if 'junior high' in name:
        types.append('junior high school')
    elif 'high' in name:
        types.append('high school')
    if 'medical' in name or 'med' in name:
        types.append('medical school')
    return types

def process_row(row):
    # Ensure 'instance of' is a list
    if isinstance(row['instance of'], str):
        try:
            instance_of_list = ast.literal_eval(row['instance of'])
        except (ValueError, SyntaxError):
            instance_of_list = []
    elif isinstance(row['instance of'], list):
        instance_of_list = row['instance of']
    else:
        instance_of_list = []

    # Extend with types from the schoolTypeMap
    instance_of_list.extend(schoolTypeMap(row['name']))
    return instance_of_list

subset_df['instance of'] = subset_df.apply(process_row, axis=1)

In [105]:
import ast

def consolidate_dataframe(df):
    """
    For each column in the DataFrame:
    - If a cell contains multiple distinct entries, it is converted into a list of unique values.
    - If all entries in a cell are identical, consolidate them into a single string.

    Parameters:
    df (pd.DataFrame): Input DataFrame.

    Returns:
    pd.DataFrame: Transformed DataFrame.
    """
    def process_cell(cell):
        # Ensure cell is properly handled
        if isinstance(cell, str):
            try:
                # Attempt to parse the string as a Python literal (e.g., list)
                parsed = ast.literal_eval(cell)
            except (ValueError, SyntaxError):
                # If it cannot be parsed, leave as a string
                return cell
        else:
            parsed = cell

        # Handle parsed data if it's a list
        if isinstance(parsed, list):
            hashable_items = []
            for item in parsed:
                if isinstance(item, dict):
                    # Convert dict to tuple of sorted items for deduplication
                    hashable_items.append(tuple(sorted(item.items())))
                else:
                    hashable_items.append(item)
            
            # Get unique values
            unique_values = list(set(hashable_items))
            
            # Convert tuples back to dictionaries, if applicable
            unique_values = [dict(item) if isinstance(item, tuple) else item for item in unique_values]
            
            return unique_values if len(unique_values) > 1 else unique_values[0] if unique_values else None

        # Return non-list cells as-is
        return cell

    # Apply the transformation to the entire DataFrame
    return df.applymap(process_cell)

# Process the DataFrame
subset_df = consolidate_dataframe(subset_df)

# Output results
print(subset_df.shape)
print(subset_df.sample(10))

/var/folders/hj/n2g00xyx4p325jm_p7rk5mx80000gn/T/ipykernel_51869/2704624317.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(process_cell)


(55177, 40)
                                                    name       wd_id  \
53345             Darrouzett Independent School District    Q5225333   
29721                              Pugu Secondary School    Q7258977   
314                            Texas Lutheran University    Q3301911   
26258                             Innovation High School   Q85769485   
25375                             Scottsboro High School   Q23302926   
8986   KSITM KATSINA STATE INSTITUTE OF TECHNOLOGY AN...  Q110826099   
31908                                       HARP Academy   Q24456710   
25515                               Fox Tech High School   Q26258685   
20007                            Monte Vista High School    Q6904948   
22464                             Versailles High School    Q7922617   

                                               altLabels  \
53345                                               None   
29721                                               None   
314            

In [106]:
subset_df.columns

Index(['name', 'wd_id', 'altLabels', 'instance of', 'wikipediaURL',
       'short name', 'nickname', 'alternative name', 'count of students',
       'image', 'seal image', 'logo image', 'small logo or icon',
       'coat of arms image', 'X username', 'Instagram username',
       'Facebook username', 'member of', 'street address',
       'coordinate location', 'headquarters location', 'postal code',
       'located in the administrative territorial entity', 'country',
       'official website', 'LinkedIn company or organization ID',
       'has subsidiary', 'part of', 'parent organization', 'owner of',
       'parentOrganizations', 'official name', 'has part(s)',
       'described at URL', 'subclass of', 'IRS Employer Identification Number',
       'school district', 'start grade', 'end grade', 'description'],
      dtype='object')

In [107]:
# drop all columns where one or zero rows have value

initial_count = subset_df.shape[1]

subset_df = subset_df.dropna(axis=1, how='all')
subset_df = subset_df.loc[:, subset_df.notna().sum() > 1]
subset_df = subset_df[['name'] + ['wd_id'] + ['altLabels'] + ['instance of'] + [col for col in subset_df.columns if col != 'name' and col != 'wd_id' and col != 'altLabels' and col != 'instance of']]

final_count = subset_df.shape[1]
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} columns without any content")

Dropping 0 columns without any content


In [108]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pandas as pd
from generate_summary import scrape_wikipedia_summary

# Function to fetch description
def fetch_description(row):
    if row.get('wikipediaURL'):
        return scrape_wikipedia_summary(row['wikipediaURL'])
    return None

# Process DataFrame in parallel
def process_in_parallel(df, workers=4):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        # Use tqdm to show progress
        results = list(tqdm(
            executor.map(fetch_description, [row for _, row in df.iterrows()]),
            total=len(df),
            desc="Fetching summaries"
        ))
    df['description'] = results
    return df

# Example usage
subset_df = process_in_parallel(subset_df, workers=8)


Fetching summaries:   0%|          | 2/55177 [00:00<6:04:36,  2.52it/s]


KeyboardInterrupt: 

In [109]:
file_name = f'processed_schools_min_cols_1733507105.361252.csv'

subset_df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

Wrote df to processed_schools_min_cols_1733507105.361252.csv


In [ ]:
# un-list items
import numpy as np

for col in subset_df.columns:
    subset_df[col] = subset_df[col].apply(
        lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x
    )

In [ ]:
print(subset_df.head())
print(subset_df.dtypes)

In [ ]:
subset_df.sample(5)

In [ ]:
subset_df['nickname'].value_counts()


subset_df.col

# Merge in nicknames

In [ ]:
file_name = 'processed_schools_min_cols_1732026738.218009.csv'

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import random

def fetch_nicknames(wikidata_id):
    query = f"""
    SELECT ?itemLabel ?altLabel WHERE {{
        BIND(wd:{wikidata_id} AS ?item)  # Bind the input entity to ?item
        OPTIONAL {{ ?item skos:altLabel ?altLabel . FILTER (LANG(?altLabel) = "en") }}
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    
    # Execute the query and fetch results
    sparql.query()
    results = sparql.query().convert()

    # Extract nicknames (altLabels) from results
    nicknames = [result.get('altLabel')['value'] for result in results['results']['bindings'] if result.get('altLabel')]
    return nicknames

def fetch_entity_with_retry(entity, max_retries=5, backoff_factor=1.5):
    retries = 0
    while retries < max_retries:
        try:
            # Replace this with the actual API call
            result = fetch_nicknames(entity)  # Your function that fetches the entity
            return result
        except Exception as e:
            # If a 429 error occurs, handle it
            if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 429:
                retries += 1
                wait_time = backoff_factor ** retries + random.uniform(0, 2)
                print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                retries += 1
                wait_time = backoff_factor ** retries + random.uniform(0, 2)
                print(f"Error getting {entity}, {e}")
                time.sleep(wait_time)
    print("Max retries reached. Skipping this entity.")
    return None  # Return None if retries are exhausted

def merge_nicknames_with_csv(csv_file, wikidata_column):
    # Read the CSV
    df = pd.read_csv(csv_file)
    
    # Create an empty list to store the nicknames
    nicknames_list = []

    # Iterate through each row and fetch the nicknames for the Wikidata ID
    for _index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Fetching Nicknames", unit="row"):
        wikidata_id = row[wikidata_column]
        nicknames = fetch_entity_with_retry(wikidata_id)
        nicknames_list.append(nicknames)

    # Add the nicknames as a new column to the DataFrame
    df['nicknames'] = nicknames_list
    
    # Save the updated DataFrame to a new CSV
    with tqdm(total=len(df), desc="Saving CSV", unit="row") as pbar:
        df.to_csv('updated_with_nicknames.csv', index=False)
        pbar.update(len(df))

In [ ]:
csv_file = file_name  # Replace with your existing CSV file path
wikidata_column = 'wd_id'  # Replace with the column containing Wikidata IDs (e.g., Q536709)

merge_nicknames_with_csv(csv_file, wikidata_column)

# Export as csv

In [ ]:
file_name = f'processed_schools_min_cols_{time.time()}.csv'

subset_df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

In [ ]:
import pandas as pd

with open('schools.csv', 'r') as file:
    df = pd.read_csv(file)

In [ ]:
df.sample(5)